In [3]:
pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.0 MB/s 
     |████████████████████████████████| 9.8 MB 40.3 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [4]:
import pmdarima as pm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
from keras.models import Sequential
from keras.layers import Dense

from tensorflow.keras.optimizers import Adam

warnings.filterwarnings('ignore')


In [5]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

deaths_df = pd.read_csv(url)

In [6]:
deaths_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/2/22,7/3/22,7/4/22,7/5/22,7/6/22,7/7/22,7/8/22,7/9/22,7/10/22,7/11/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7723,7724,7725,7725,7725,7727,7728,7728,7728,7728
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3501,3502,3502,3502,3506,3508,3510,3512,3514,3514
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6875,6875,6875,6875,6875,6875,6875,6875,6875,6875
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,153,153,153,153,153,153,153,153,153,153
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1900,1900,1900,1900,1900,1900,1900,1900,1900,1900


In [7]:
deaths_df.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '7/2/22', '7/3/22', '7/4/22', '7/5/22', '7/6/22', '7/7/22', '7/8/22',
       '7/9/22', '7/10/22', '7/11/22'],
      dtype='object', length=906)

In [8]:
df = deaths_df.loc[:,'1/22/20':]
df.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,7/2/22,7/3/22,7/4/22,7/5/22,7/6/22,7/7/22,7/8/22,7/9/22,7/10/22,7/11/22
0,0,0,0,0,0,0,0,0,0,0,...,7723,7724,7725,7725,7725,7727,7728,7728,7728,7728
1,0,0,0,0,0,0,0,0,0,0,...,3501,3502,3502,3502,3506,3508,3510,3512,3514,3514
2,0,0,0,0,0,0,0,0,0,0,...,6875,6875,6875,6875,6875,6875,6875,6875,6875,6875
3,0,0,0,0,0,0,0,0,0,0,...,153,153,153,153,153,153,153,153,153,153
4,0,0,0,0,0,0,0,0,0,0,...,1900,1900,1900,1900,1900,1900,1900,1900,1900,1900


In [9]:
df = df.transpose()

In [11]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,275,276,277,278,279,280,281,282,283,284
7/7/22,7727,3508,6875,153,1900,0,142,129109,8629,81,...,7351,1637,14,5738,43089,5662,0,2149,4008,5560
7/8/22,7728,3510,6875,153,1900,0,143,129109,8629,82,...,7351,1637,14,5739,43089,5662,0,2149,4008,5562
7/9/22,7728,3512,6875,153,1900,0,143,129109,8629,82,...,7351,1637,14,5740,43089,5662,0,2149,4008,5563
7/10/22,7728,3514,6875,153,1900,0,143,129145,8629,84,...,7351,1637,14,5741,43089,5662,0,2149,4008,5564
7/11/22,7728,3514,6875,153,1900,0,143,129145,8629,83,...,7373,1637,14,5741,43089,5662,0,2149,4008,5565


In [12]:
df = df.sum(axis=1)

In [15]:
df.head()

1/22/20    17
1/23/20    18
1/24/20    26
1/25/20    42
1/26/20    56
dtype: int64